In [26]:
%%timeit
import enchant
from enchant.checker import SpellChecker
import pylanguagetool
from autocorrect import spell
import re
#pylanguagetool pylanguagetool textfile.txt
#spell('gff') out:off

1.8 µs ± 21.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


#### Заменяет "испорченное" слово на [MASK]

In [40]:
def pyenchant_mask(text):
    import enchant
    from enchant.checker import SpellChecker
    chkr = SpellChecker("ru_RU",text)
    for err in chkr:
            err.replace('[MASK]')   
    text = chkr.get_text()
    return text

#### Находит и корректирует слово 

In [1]:
def pyenchant_autocorrect(text):
    #import enchant
    #from autocorrect import spell
    #from enchant.checker import SpellChecker
    chkr = SpellChecker("ru_RU",text)
    for err in chkr:
        lul = spell(err)
        err.replace(lul)
    text = chkr.get_text()
    return text

In [7]:
#Сreating russian dictionary for code
import re
with open("dict.opcorpora.txt") as f:  #file from open corpora, 5890204 words
    slovar = f.readlines()
dictionary = []
regex = re.compile('[^а-яА-ЯёЁ-]')      #deleting non-cyryllic symbols, for clear dictionary  
for i in slovar:
    i = regex.sub('',i)
    dictionary.append(i.lower())

### Используемый текст для эксперимента

In [8]:
text = '33-летний муж-чина пред-стал перед судом за то, что на-нес тяжкие телес-ные повреждения 36-летней Динаре Акамбаевой, ко-торая работала на том же предприятии. Жен щина от побоев скончалась.2 января 2019 года Акамбаева за-держалась на работе, ожидая дежурную машину. Тогда Утешкенов подвез ее, но в об щежитие, г де она пр-оживала во время вахты, женщина так и не при-ехала.'
#def text_maker(text):

#text1 = []
#for i in text:
    #text1.append(i.split(' '))

In [9]:
text

'33-летний муж-чина пред-стал перед судом за то, что на-нес тяжкие телес-ные повреждения 36-летней Динаре Акамбаевой, ко-торая работала на том же предприятии. Жен щина от побоев скончалась.2 января 2019 года Акамбаева за-держалась на работе, ожидая дежурную машину. Тогда Утешкенов подвез ее, но в об щежитие, г де она пр-оживала во время вахты, женщина так и не при-ехала.'

#### Функция для удаления дефисов

In [10]:
def hyphen_remover(text):
    text = text.split(' ')
    word_vector = []
    new_text = ''
    for word in text:
        if '-' in word and not any(digit.isdigit() for digit in word) and word.lower() not in (words for words in dictionary): #checking - and digits in word
            word = word.replace('-','')
        word_vector.append(word)
    #**************************************
    for word in word_vector:
        new_text += word+' '
    return new_text       

#### Результат ее работы

In [12]:
g

'33-летний мужчина предстал перед судом за то, что нанес тяжкие телесные повреждения 36-летней Динаре Акамбаевой, которая работала на том же предприятии. Жен щина от побоев скончалась.2 января 2019 года Акамбаева задержалась на работе, ожидая дежурную машину. Тогда Утешкенов подвез ее, но в об щежитие, г де она проживала во время вахты, женщина так и не приехала. '

In [167]:
#    alphabet = ['Й','Ц','У','К','Е','Н','Г','Ш','Щ','З','Х','Ф','Ы','В','А','П','Р','О','Л','Д','Ж','Э','Я','Ч','С','М','И','Т','Б','Ю']
#    g_vec = g.split(' ') # вектор слов из текста без масок для сравнивания
#    t_vec = t.split(' ') # вектор слов из текста с масками
    #********************************  
#    for i in range(len(t_vec)):
#        if '[MASK]' in t_vec[i] and any((letter in g_vec[i]) for letter in alphabet) and '.' not in g_vec[i-1]:
#            text = text.replace(g_vec[i-1]+' '+'[MASK]',g_vec[i-1]+' '+ g_vec[i])
#        elif '[MASK]' in t_vec[i] and (g_vec[i-1]+g_vec[i]).lower() in dictionary:
#            text = text.replace(t_vec[i-1]+' '+'[MASK]',g_vec[i-1] + g_vec[i])
#        elif '[MASK]' in t_vec[i] and (g_vec[i]+g_vec[i+1]).lower() in dictionary:
#            text = text.replace('[MASK]'+' '+t_vec[i+1],g_vec[i] + g_vec[i+1])
    #********************************
#    return text                             

#### Небольшой препроцессинг

In [168]:
t = re.sub("[^\w,]",' ', str(g))

In [13]:
text = g.split(' ')
word_vector = []
new_text = ''
#*******************************************************************************************************************
for i in range(len(text)-1):
    if text[i] not in dictionary and not any(digit.isdigit() for digit in text[i]):
        text[i] = text[i]+text[i+1]
        i+=1
    word_vector.append(text[i])

#### Замаскируем слова с ошибками

In [ ]:
t = pyenchant_mask(g)

In [108]:
t

'33-летний мужчина предстал перед судом за то, что нанес тяжкие телесные повреждения 36-летней Динаре [MASK], которая работала на том же предприятии. Жен [MASK] от побоев скончалась.2 января 2019 года [MASK] задержалась на работе, ожидая дежурную машину. Тогда [MASK] подвез ее, но в об [MASK], [MASK] де она проживала во время вахты, женщина так и не приехала. '

#### Пробегая по словарю и сравниваня g_vec и t_vec получим исправление

In [174]:
#g=hyphen_remover(text)
#t = pyenchant_mask(g)
alphabet = ['Й','Ц','У','К','Е','Н','Г','Ш','Щ','З','Х','Ф','Ы','В','А','П','Р','О','Л','Д','Ж','Э','Я','Ч','С','М','И','Т','Б','Ю']
final_vec = []
g_vec = g.split(' ') # вектор слов из текста без масок для сравнивания
t_vec = t.split(' ') # вектор слов из текста с масками
for i in range(len(t_vec)):
    if '[MASK]' in t_vec[i] and any((letter in g_vec[i]) for letter in alphabet) and '.' not in g_vec[i-1]:
        t = t.replace(g_vec[i-1]+' '+'[MASK]',g_vec[i-1]+' '+ g_vec[i])
    elif '[MASK]' in t_vec[i] and (g_vec[i-1]+g_vec[i]).lower() in dictionary:
        t = t.replace(t_vec[i-1]+' '+'[MASK]',g_vec[i-1] + g_vec[i])
    elif '[MASK]' in t_vec[i] and (g_vec[i]+g_vec[i+1]).lower() in dictionary:
        t = t.replace('[MASK]'+' '+t_vec[i+1],g_vec[i] + g_vec[i+1])

In [170]:
t

'33-летний мужчина предстал перед судом за то, что нанес тяжкие телесные повреждения 36-летней Динаре Акамбаевой,, которая работала на том же предприятии. Женщина от побоев скончалась.2 января 2019 года Акамбаева задержалась на работе, ожидая дежурную машину. Тогда Утешкенов подвез ее, но в общежитии, где она проживала во время вахты, женщина так и не приехала. '